In [1]:
import pandas as pd
import polars as pl
import dask.dataframe as dd
import dask.multiprocessing
import xarray as xr

In [2]:
bathy = xr.open_dataset("/home/an1n23/datasets/IBCSO_v2_ice-surface_WGS84.nc")

In [15]:
def get_depth(row):
    return bathy.z.interp(lat=row['LATITUDE (degree_north)'], lon=row['LONGITUDE (degree_east)']).item()

def get_depth_nearest(row):
    return bathy.z.sel(lat=row['LATITUDE (degree_north)'], lon=row['LONGITUDE (degree_east)'], method="nearest").item()

In [4]:
!ls /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/*.part > arbit_filelist_argo.txt

In [5]:
argofiles = pd.read_csv("arbit_filelist_argo.txt", header=None)

In [6]:
arr = pd.read_csv(argofiles[0][0])

In [7]:
arr

,Unnamed: 0,PLATFORM_CODE,DATE (YYYY-MM-DDTHH:MI:SSZ),LATITUDE (degree_north),LONGITUDE (degree_east),POSITION_QC,PRES_ADJUSTED (decibar),PRES_ADJUSTED_QC,TEMP_ADJUSTED (degree_Celsius),TEMP_ADJUSTED_QC,PSAL_ADJUSTED (psu),PSAL_ADJUSTED_QC
0,0,1900044,2006-04-29 19:12:31+00:00,-48.76200,131.05701,1,4.20,1.0,9.134,1.0,34.496,1.0
1,1,1900044,2006-04-29 19:12:31+00:00,-48.76200,131.05701,1,5.70,1.0,9.141,1.0,34.497,1.0
2,2,1900044,2006-04-29 19:12:31+00:00,-48.76200,131.05701,1,15.80,1.0,9.142,1.0,34.497,1.0
3,3,1900044,2006-04-29 19:12:31+00:00,-48.76200,131.05701,1,25.50,1.0,9.144,1.0,34.496,1.0
4,4,1900044,2006-04-29 19:12:31+00:00,-48.76200,131.05701,1,35.80,1.0,9.145,1.0,34.497,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9212387,9212882,3901216,2021-03-11 22:40:32+00:00,-46.53785,-80.01684,1,453.96,1.0,5.527,1.0,NaN,NaN
9212388,9212883,3901216,2021-03-11 22:40:32+00:00,-46.53785,-80.01684,1,456.00,1.0,5.524,1.0,NaN,NaN
9212389,9212884,3901216,2021-03-11 22:40:32+00:00,-46.53785,-80.01684,1,458.00,1.0,5.522,1.0,NaN,NaN
9212390,9212885,3901216,2021-03-11 22:40:32+00:00,-46.53785,-80.01684,1,460.00,1.0,5.528,1.0,NaN,NaN


In [8]:
unique_coords = arr.drop_duplicates(subset=['LATITUDE (degree_north)', 'LONGITUDE (degree_east)'])\
                                       [['LATITUDE (degree_north)', 'LONGITUDE (degree_east)']]

In [9]:
unique_coords

,LATITUDE (degree_north),LONGITUDE (degree_east)
0,-48.76200,131.05701
51,-48.87500,132.48500
102,-49.50200,134.48300
170,-48.64200,136.46201
221,-48.30000,137.07001
...,...,...
9209440,-46.84642,-80.60884
9210008,-46.91248,-80.44653
9210580,-46.90984,-80.41051
9211583,-46.81642,-80.07797


In [19]:
unique_coords["ECHODEPTH"] = unique_coords.apply(get_depth_nearest, axis=1)

In [20]:
unique_coords

,LATITUDE (degree_north),LONGITUDE (degree_east),ECHODEPTH
0,-48.76200,131.05701,-3491.0
51,-48.87500,132.48500,-3379.0
102,-49.50200,134.48300,-3179.0
170,-48.64200,136.46201,-3118.0
221,-48.30000,137.07001,-3009.0
...,...,...,...
9209440,-46.84642,-80.60884,-4082.0
9210008,-46.91248,-80.44653,-4072.0
9210580,-46.90984,-80.41051,-4074.0
9211583,-46.81642,-80.07797,-4031.0


In [27]:
arr = arr.merge(unique_coords, on=['LATITUDE (degree_north)', 'LONGITUDE (degree_east)'], how='left')

In [32]:
ind=51
arr.iloc[ind][['LATITUDE (degree_north)', 'LONGITUDE (degree_east)', "ECHODEPTH"]]

LATITUDE (degree_north)    -48.875
LONGITUDE (degree_east)    132.485
ECHODEPTH                  -3379.0
Name: 51, dtype: object

In [17]:
!mkdir /scratch/an1n23/argo_RS_AS

In [33]:
del arr

In [34]:
for f in argofiles[0]:
    print("Processing file:", f)
    
    argo = pd.read_csv(f)
    unnamed_cols = [col for col in argo.columns if col.startswith("Unnamed")]
    argo = argo.drop(columns = unnamed_cols)
    sel_lat = argo["LATITUDE (degree_north)"] <= -50
    
    argo = argo.loc[sel_lat]

    unique_coords = argo.drop_duplicates(subset=['LATITUDE (degree_north)', 'LONGITUDE (degree_east)'])\
                                       [['LATITUDE (degree_north)', 'LONGITUDE (degree_east)']]
    unique_coords["ECHODEPTH"] = unique_coords.apply(get_depth_nearest, axis=1)
    
    argo = argo.merge(unique_coords, on=['LATITUDE (degree_north)', 'LONGITUDE (degree_east)'], how='left')
    
    argo.to_csv("/scratch/an1n23/argo_RS_AS/"+f.split("/")[-1], index=False)
    del(argo)
        
        
    

Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/00.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/01.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/02.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/03.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/04.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/05.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/06.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/07.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/08.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/09.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/10.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/11.part
Processing file: /home/an1n23/datasets/merged_RS_AS_ARGO29082023.csv/12.part

In [35]:
! echo "completed argo assign bathy 02102023" > arbit_assignBathyArgo.txt